In [4]:
import os
PROJECT = "qwiklabs-gcp-ml-49b827b781ab" # REPLACE WITH YOUR PROJECT ID
BUCKET = "qwiklabs-gcp-ml-49b827b781ab" # REPLACE WITH YOUR BUCKET NAME
REGION = "us-central1" # REPLACE WITH YOUR BUCKET REGION e.g. us-central1
MODEL_TYPE = "dnn_dropout"  # "linear", "dnn", "dnn_dropout", or "cnn"

# Do not change these
os.environ["PROJECT"] = PROJECT
os.environ["BUCKET"] = BUCKET
os.environ["REGION"] = REGION
os.environ["MODEL_TYPE"] = MODEL_TYPE
os.environ["TFVERSION"] = "1.13"  # Tensorflow version

In [5]:
%%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

Updated property [core/project].
Updated property [compute/region].


# Local Train
### 아래쪽 cell에는 GPU에서 실행하는 방법이 있다.

In [6]:
%%bash
rm -rf mnistmodel.tar.gz mnist_trained
gcloud ml-engine local train \
    --module-name=trainer.task \
    --package-path=${PWD}/mnistmodel/trainer \
    -- \
    --output_dir=${PWD}/mnist_trained_$MODEL_TYPE \
    --train_steps=100 \
    --learning_rate=0.01 \
    --model=$MODEL_TYPE

Extracting mnist/data/train-images-idx3-ubyte.gz
Extracting mnist/data/train-labels-idx1-ubyte.gz
Extracting mnist/data/t10k-images-idx3-ubyte.gz
Extracting mnist/data/t10k-labels-idx1-ubyte.gz





Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Instructions for updating:
Please use tf.data to implement this functionality.
Instructions for updating:
Please use tf.one_hot on tensors.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


INFO:tensorflow:TF_CONFIG environment variable: {u'environment': u'cloud', u'cluster': {}, u'job': {u'args': [u'--output_dir=/home/jupyter/20191205/07_AI_platform_training_mnist/mnist_trained_dnn_dropout', u'--train_steps=100', u'--learning_rate=0.01', u'--model=dnn_dropout'], u'job_name': u'trainer.task'}, u'task': {}}
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_task_type': 'wor

# Train on GPU (--scale-tier=BASIC_GPU)

In [4]:
%%bash
OUTDIR=gs://${BUCKET}/mnist/trained_${MODEL_TYPE}
JOBNAME=mnist_${MODEL_TYPE}_$(date -u +%y%m%d_%H%M%S)
echo $OUTDIR $REGION $JOBNAME
gsutil -m rm -rf $OUTDIR
gcloud ml-engine jobs submit training $JOBNAME \
    --region=$REGION \
    --module-name=trainer.task \
    --package-path=${PWD}/mnistmodel/trainer \
    --job-dir=$OUTDIR \
    --staging-bucket=gs://$BUCKET \
    --scale-tier=BASIC_GPU \
    --runtime-version=$TFVERSION \
    -- \
    --output_dir=$OUTDIR \
    --train_steps=10000 --learning_rate=0.01 --train_batch_size=512 \
    --model=$MODEL_TYPE --batch_norm

gs://qwiklabs-gcp-ml-49b827b781ab/mnist/trained_dnn us-central1 mnist_dnn_191207_034203
jobId: mnist_dnn_191207_034203
state: QUEUED


Removing gs://qwiklabs-gcp-ml-49b827b781ab/mnist/trained_dnn/#1575689481578950...
Removing gs://qwiklabs-gcp-ml-49b827b781ab/mnist/trained_dnn/checkpoint#1575689483540599...
Removing gs://qwiklabs-gcp-ml-49b827b781ab/mnist/trained_dnn/eval/#1575689486666606...
Removing gs://qwiklabs-gcp-ml-49b827b781ab/mnist/trained_dnn/eval/events.out.tfevents.1575689486.cmle-training-12968357840110515339#1575689488104453...
Removing gs://qwiklabs-gcp-ml-49b827b781ab/mnist/trained_dnn/events.out.tfevents.1575689440.cmle-training-12968357840110515339#1575689496835283...
Removing gs://qwiklabs-gcp-ml-49b827b781ab/mnist/trained_dnn/export/#1575689489193303...
Removing gs://qwiklabs-gcp-ml-49b827b781ab/mnist/trained_dnn/export/exporter/#1575689489427489...
Removing gs://qwiklabs-gcp-ml-49b827b781ab/mnist/trained_dnn/export/exporter/1575689488/#1575689494800468...
Removing gs://qwiklabs-gcp-ml-49b827b781ab/mnist/trained_dnn/export/exporter/1575689488/saved_model.pb#1575689495120796...
Removing gs://qwiklab

# Monitoring training with TensorBoard

In [ ]:
from google.datalab.ml import TensorBoard
TensorBoard().start("gs://{}/mnist/trained_{}".format(BUCKET, MODEL_TYPE))

In [ ]:
for pid in TensorBoard.list()["pid"]:
    TensorBoard().stop(pid)
    print("Stopped TensorBoard with pid {}".format(pid))